# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

## Loading the dataset

In [2]:
combined = pd.read_csv("../final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2086 entries, 0 to 2085
Data columns (total 90 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          2086 non-null   int64  
 1   match_id                            2086 non-null   object 
 2   goals                               2086 non-null   float64
 3   assists                             2086 non-null   float64
 4   direct_goal_contributions           2086 non-null   float64
 5   non_penalty_goals                   2086 non-null   float64
 6   penalties_scored                    2086 non-null   float64
 7   penalties_attempted                 2086 non-null   float64
 8   yellow_cards                        2086 non-null   float64
 9   red_cards                           2086 non-null   float64
 10  expected_goals                      2086 non-null   float64
 11  non_penalty_expected_goals          2086 no

In [4]:
combined.head()

,Unnamed: 0,match_id,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,...,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,0,m-00381,0.720489,0.165106,0.885596,0.656287,0.064203,0.076105,0.395800,0.073663,...,6,4,2,5,11,8,2,1,0,0
1,1,m-00382,0.085780,-0.107836,-0.022056,0.044580,0.041200,0.064124,0.226430,-0.000329,...,4,1,7,4,11,9,1,1,0,0
2,2,m-00383,0.411290,0.245491,0.656781,0.298064,0.113226,0.121619,0.429298,0.001779,...,6,9,5,5,9,11,1,2,0,0
3,3,m-00384,-0.085340,-0.143024,-0.228364,-0.078692,-0.006648,-0.006648,-0.068740,0.029937,...,1,4,2,5,9,8,2,1,0,0
4,4,m-00385,-0.240018,0.037276,-0.202742,-0.250471,0.010452,0.017316,-0.037952,-0.034174,...,2,5,3,5,11,12,2,2,0,0


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
match_columns = [
	"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
stats_columns = [
	"goals","assists","direct_goal_contributions","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"penalties_made","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","progressive_passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]
player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
pure_stats_columns = ["minutes_played"] + stats_columns
team_stats_columns = ["team_id"] + stats_columns


## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[stats_columns])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = 69
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


2024-01-30 15:40:54.538893: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


34/34 [==============================] - 0s 425us/step - loss: 28.0244 - accuracy: 0.3747
Epoch 2/500
34/34 [==============================] - 0s 324us/step - loss: 15.7965 - accuracy: 0.4706
Epoch 3/500
34/34 [==============================] - 0s 335us/step - loss: 12.4271 - accuracy: 0.4730
Epoch 4/500
34/34 [==============================] - 0s 338us/step - loss: 9.7817 - accuracy: 0.4694
Epoch 5/500
34/34 [==============================] - 0s 343us/step - loss: 9.1566 - accuracy: 0.4694
Epoch 6/500
34/34 [==============================] - 0s 330us/step - loss: 8.9048 - accuracy: 0.4676
Epoch 7/500
34/34 [==============================] - 0s 327us/step - loss: 8.7506 - accuracy: 0.4724
Epoch 8/500
34/34 [==============================] - 0s 326us/step - loss: 8.7100 - accuracy: 0.4688
Epoch 9/500
34/34 [==============================] - 0s 330us/step - loss: 8.9118 - accuracy: 0.4688
Epoch 10/500
34/34 [==============================] - 0s 324us/step - loss: 8.7473 - accuracy: 0.468

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 323us/step
[ 1.5596617   1.2908837  13.315933   11.986653    4.7053537   4.1912384
  5.734507    5.1445947  10.553586   10.463669    1.3018817   1.6724641
  0.10219375 -0.05119775]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 481us/step - loss: 25.6436 - accuracy: 0.4245
Epoch 2/500
34/34 [==============================] - 0s 400us/step - loss: 14.2073 - accuracy: 0.4694
Epoch 3/500
34/34 [==============================] - 0s 360us/step - loss: 12.1265 - accuracy: 0.4688
Epoch 4/500
34/34 [==============================] - 0s 557us/step - loss: 10.7531 - accuracy: 0.4694
Epoch 5/500
34/34 [==============================] - 0s 386us/step - loss: 9.9014 - accuracy: 0.4694
Epoch 6/500
34/34 [==============================] - 0s 359us/step - loss: 9.3841 - accuracy: 0.4676
Epoch 7/500
34/34 [==============================] - 0s 398us/step - loss: 9.3428 - accuracy: 0.4712
Epoch 8/500
34/34 [==============================] - 0s 346us/step - loss: 9.1996 - accuracy: 0.4724
Epoch 9/500
34/34 [==============================] - 0s 566us/step - loss: 8.8169 - accuracy: 0.4676
Epoch 10/500
34/34 [==============================] - 0s 369us/step - loss: 8.8082 - ac

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 452us/step


[ 1.0841223   1.0223368  13.856777   10.02816     4.0679746   3.4104197
  6.1687503   4.81867    10.740551   10.028124    1.2110811   1.873328
  0.24459563  0.32893485]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 460us/step - loss: 24.8978 - accuracy: 0.4472
Epoch 2/500
34/34 [==============================] - 0s 376us/step - loss: 13.8071 - accuracy: 0.4706
Epoch 3/500
34/34 [==============================] - 0s 553us/step - loss: 11.7124 - accuracy: 0.4694
Epoch 4/500
34/34 [==============================] - 0s 361us/step - loss: 10.6490 - accuracy: 0.4682
Epoch 5/500
34/34 [==============================] - 0s 410us/step - loss: 9.9911 - accuracy: 0.4748
Epoch 6/500
34/34 [==============================] - 0s 347us/step - loss: 9.3923 - accuracy: 0.4700
Epoch 7/500
34/34 [==============================] - 0s 313us/step - loss: 9.1199 - accuracy: 0.4712
Epoch 8/500
34/34 [==============================] - 0s 332us/step - loss: 9.3929 - accuracy: 0.4718
Epoch 9/500
34/34 [==============================] - 0s 347us/step - loss: 8.8511 - accuracy: 0.4760
Epoch 10/500
34/34 [==============================] - 0s 342us/step - loss: 8.7843 - ac

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 342us/step
[-0.2999388   1.551403    7.3919826  14.110368    1.4917399   4.233017
  4.3964047   6.0553565  12.242245   10.601787    1.4038912   1.650486
  0.20001292 -0.18333419]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 424us/step - loss: 27.1940 - accuracy: 0.4376
Epoch 2/500
34/34 [==============================] - 0s 366us/step - loss: 13.8679 - accuracy: 0.4706
Epoch 3/500
34/34 [==============================] - 0s 341us/step - loss: 11.6975 - accuracy: 0.4736
Epoch 4/500
34/34 [==============================] - 0s 344us/step - loss: 10.6128 - accuracy: 0.4694
Epoch 5/500
34/34 [==============================] - 0s 339us/step - loss: 9.8242 - accuracy: 0.4706
Epoch 6/500
34/34 [==============================] - 0s 338us/step - loss: 9.4361 - accuracy: 0.4730
Epoch 7/500
34/34 [==============================] - 0s 345us/step - loss: 9.0713 - accuracy: 0.4724
Epoch 8/500
34/34 [==============================] - 0s 336us/step - loss: 9.0574 - accuracy: 0.4736
Epoch 9/500
34/34 [==============================] - 0s 341us/step - loss: 8.7952 - accuracy: 0.4736
Epoch 10/500
34/34 [==============================] - 0s 338us/step - loss: 8.8437 - ac

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 356us/step
[ 0.99058056  1.3813982  15.223371   10.699188    3.779078    4.0946
  5.5960445   5.051801   10.133969    8.568032    1.2757156   2.0458
  0.06627411  0.14096297]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 478us/step - loss: 26.6809 - accuracy: 0.3771
Epoch 2/500
34/34 [==============================] - 0s 392us/step - loss: 13.3674 - accuracy: 0.4718
Epoch 3/500
34/34 [==============================] - 0s 371us/step - loss: 11.5276 - accuracy: 0.4676
Epoch 4/500
34/34 [==============================] - 0s 361us/step - loss: 10.5024 - accuracy: 0.4712
Epoch 5/500
34/34 [==============================] - 0s 333us/step - loss: 9.8450 - accuracy: 0.4742
Epoch 6/500
34/34 [==============================] - 0s 335us/step - loss: 9.4377 - accuracy: 0.4700
Epoch 7/500
34/34 [==============================] - 0s 339us/step - loss: 9.1984 - accuracy: 0.4694
Epoch 8/500
34/34 [==============================] - 0s 498us/step - loss: 9.7681 - accuracy: 0.4742
Epoch 9/500
34/34 [==============================] - 0s 355us/step - loss: 8.8965 - accuracy: 0.4712
Epoch 10/500
34/34 [==============================] - 0s 333us/step - loss: 8.8234 - ac

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 325us/step
[ 1.3230698   0.7321819  11.987978   10.010966    3.7661908   3.2692194
  6.028605    4.985801   10.435602    6.825079    1.0476061   1.546419
  0.21520494  0.08806247]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1229,0,2,5,16,1,5,1,9,9,12,1,2,0,0
1597,2,2,19,10,4,6,5,6,9,11,2,0,0,0
2015,1,2,18,15,8,5,9,3,11,13,0,4,0,0
25,1,1,11,18,2,6,5,9,13,8,1,2,0,0
1628,1,0,12,14,3,4,8,4,13,12,3,3,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[stats_columns]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)
print(X_train)

         goals   assists  direct_goal_contributions  non_penalty_goals  \
1276  0.366503 -0.022074                   0.344428           0.371299   
1570 -1.117631 -0.519741                  -1.637373          -0.934464   
864  -0.660156  0.036074                  -0.624082          -0.481211   
1382 -0.499839 -0.325765                  -0.825604          -0.535268   
20   -0.390848 -0.137376                  -0.528224          -0.276079   
...        ...       ...                        ...                ...   
1542  0.715649  0.210350                   0.925999           0.618118   
2062 -0.307234 -0.307463                  -0.614697          -0.252090   
479  -0.156351  0.093343                  -0.063009          -0.169494   
561   0.409117  0.230599                   0.639716           0.274980   
1785  0.059156 -0.117956                  -0.058800           0.107958   

      penalties_scored  penalties_attempted  yellow_cards  red_cards  \
1276         -0.004796            -0.06

#### Scaling X and y

In [26]:
whole_x_scaler = StandardScaler(copy=True).fit(X)
whole_x_train = whole_x_scaler.transform(X)
dump(whole_x_scaler, '../prediction_scaler.bin')


# y_scaler = StandardScaler(copy=True).fit(y_train)

X_scaler = StandardScaler(copy=True).fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
n=15
print(X_train)
pca = PCA(n_components = n, random_state=576)
pca.fit(X)
feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
feature_to_pc_map.to_csv("../feature_to_15_pcs.csv")

dump(pca, '../prediction_pca.bin')
whole_X_pca = pca.transform(whole_x_train)

pca = PCA(n_components = n, random_state=576)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)


feature_to_pc_map

[[ 0.78622664 -0.0438009   0.48421519 ...  1.16024711  1.3415394
   0.17933687]
 [-2.27468291 -1.65931122 -2.12541236 ...  0.44404888 -1.17566338
  -0.02129511]
 [-1.331175    0.14495778 -0.79111497 ...  0.58120935  1.37025823
  -0.59268471]
 ...
 [-0.29211804  0.33086187 -0.05229599 ...  0.20081513  0.96211483
   0.02711332]
 [ 0.87411575  0.77641947  0.87304918 ... -0.30134203  0.28582233
  -0.77632857]
 [ 0.15234919 -0.35504828 -0.04675348 ... -0.37719573  1.56336307
   1.03168733]]


/opt/homebrew/anaconda3/envs/pl-stats/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
0,0.000136,0.000104,0.000240,0.000124,0.000012,0.000015,0.000184,6.751298e-06,0.000137,0.000125,...,0.002729,0.000062,0.000238,0.000759,0.000524,0.000055,0.000023,0.000018,0.000003,0.000001
1,0.000681,0.000437,0.001118,0.000623,0.000058,0.000073,0.000368,1.625723e-05,0.000653,0.000595,...,0.001997,-0.000150,-0.003450,-0.010844,-0.007449,-0.000737,-0.000311,-0.000236,-0.000055,-0.000020
2,0.001282,0.000729,0.002012,0.001216,0.000067,0.000083,0.000056,1.705475e-06,0.001194,0.001129,...,-0.008349,0.000083,0.002450,0.007679,0.005277,0.000495,0.000222,0.000171,0.000034,0.000016
3,-0.003376,-0.001690,-0.005066,-0.002888,-0.000488,-0.000601,-0.002642,-4.050176e-05,-0.003454,-0.002977,...,-0.022383,-0.000098,0.000566,0.002551,0.001994,0.000602,0.000140,0.000100,0.000023,0.000017
4,0.004766,0.001218,0.005983,0.004098,0.000667,0.000884,-0.000134,1.207533e-04,0.004436,0.003735,...,-0.035071,0.000274,0.006460,0.020095,0.013786,0.001689,0.000667,0.000528,0.000082,0.000057
5,0.001983,0.003533,0.005515,0.001500,0.000483,0.000623,-0.003870,-2.605263e-04,0.001035,0.000543,...,-0.169014,-0.000961,0.006441,0.020169,0.013793,0.001634,0.000385,0.000200,0.000156,0.000029
6,0.008449,0.003683,0.012132,0.007883,0.000566,0.000711,-0.003268,-5.348711e-05,0.007120,0.006551,...,-0.038547,0.000088,0.009144,0.032836,0.023840,0.004752,0.001242,0.001125,0.000104,0.000014
7,0.003757,0.004940,0.008697,0.003073,0.000684,0.000906,0.002760,-5.405054e-07,0.003590,0.002872,...,0.081711,0.000292,-0.004296,-0.014378,-0.010192,-0.003533,-0.000367,-0.000267,-0.000086,-0.000014
8,0.013005,0.003720,0.016725,0.011337,0.001668,0.002039,-0.001885,-2.102942e-04,0.012971,0.011359,...,0.069360,-0.001586,-0.027144,-0.088380,-0.061637,-0.004762,-0.002945,-0.002402,-0.000405,-0.000138
9,-0.015551,-0.000269,-0.015820,-0.013421,-0.002129,-0.002477,-0.005907,-1.644853e-04,-0.016618,-0.014646,...,0.066142,-0.002062,-0.035580,-0.111469,-0.076272,-0.006374,-0.003702,-0.003011,-0.000534,-0.000157


The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=stats_columns)
df.index.name = "principle_component_number"
df

,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,-0.116047,-0.126931,-0.125582,-0.116348,-0.072409,-0.073995,-0.131690,-0.080417,-0.120351,-0.121297,...,-0.114109,-0.112001,-0.051202,-0.052231,-0.052423,-0.051430,-0.050104,-0.050101,-0.035660,-0.024917
1,-0.101985,-0.073905,-0.095093,-0.096727,-0.096736,-0.099138,0.008275,-0.005910,-0.099353,-0.093691,...,0.042869,0.154562,0.292582,0.294916,0.294425,0.269105,0.287929,0.273591,0.244782,0.174145
2,0.199563,0.087974,0.163102,0.186055,0.208590,0.209463,-0.095288,-0.063566,0.180304,0.165189,...,-0.192180,-0.029430,0.164484,0.165248,0.165113,0.160261,0.160663,0.155778,0.130833,0.085010
3,0.057164,-0.093592,-0.001468,-0.014361,0.465294,0.452594,0.057931,0.050494,0.060764,-0.008264,...,0.129481,0.061230,-0.023009,-0.022061,-0.021136,-0.018713,-0.037196,-0.029016,-0.040945,-0.051665
4,-0.186367,0.091399,-0.081914,-0.226855,0.123512,0.115677,-0.052364,-0.282789,-0.192915,-0.233649,...,-0.242698,-0.097850,0.021926,0.031689,0.034859,0.009735,0.035985,0.012342,0.056691,0.133737
5,0.055562,0.134021,0.089840,0.072935,-0.068608,-0.074232,-0.126563,-0.239930,0.013357,0.027134,...,-0.003667,-0.044767,-0.018579,-0.022718,-0.023739,0.007291,-0.046592,0.003387,0.004261,-0.460018
6,-0.027585,-0.089155,-0.053778,-0.034441,0.023458,0.009692,0.081878,-0.193080,-0.005903,-0.008247,...,0.030190,-0.039972,0.100200,0.069456,0.053156,-0.054704,0.105198,0.178814,0.230872,-0.778886
7,-0.045888,0.054972,-0.006999,-0.046843,-0.023618,-0.007733,-0.025292,0.877792,-0.062757,-0.068587,...,-0.127032,-0.024093,0.043621,0.033199,0.028623,0.026565,0.040596,0.090206,-0.019019,-0.255244
8,-0.103178,-0.084647,-0.100213,-0.143742,0.177162,0.152405,-0.070860,0.089740,-0.079778,-0.113749,...,-0.166506,-0.030804,-0.047646,-0.040137,-0.037993,0.139004,-0.054866,-0.087049,0.128764,-0.102808


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(15, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500


67/67 [==============================] - 0s 346us/step - loss: 33.0088 - accuracy: 0.2122
Epoch 2/500
67/67 [==============================] - 0s 310us/step - loss: 24.8388 - accuracy: 0.2620
Epoch 3/500
67/67 [==============================] - 0s 285us/step - loss: 23.2700 - accuracy: 0.2758
Epoch 4/500
67/67 [==============================] - 0s 292us/step - loss: 22.6146 - accuracy: 0.2710
Epoch 5/500
67/67 [==============================] - 0s 284us/step - loss: 22.4010 - accuracy: 0.2782
Epoch 6/500
67/67 [==============================] - 0s 283us/step - loss: 22.4356 - accuracy: 0.2878
Epoch 7/500
67/67 [==============================] - 0s 286us/step - loss: 22.2272 - accuracy: 0.2818
Epoch 8/500
67/67 [==============================] - 0s 287us/step - loss: 22.1326 - accuracy: 0.2764
Epoch 9/500
67/67 [==============================] - 0s 305us/step - loss: 22.2538 - accuracy: 0.2824
Epoch 10/500
67/67 [==============================] - 0s 292us/step - loss: 22.1456 - accuracy

#### Testing

In [33]:
y_hat = model.predict(X_test)

14/14 [==============================] - 0s 315us/step


In [34]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [35]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) == home_shots+away_shots else 0
	total_shots_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots else 0
	total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if np.floor(home_goals_hat) == home_goals and np.floor(away_goals_hat) == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) == home_fouls + away_fouls else 0
	total_fouls_over += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0

	total_corners += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) == home_corners + away_corners else 0
	total_corners_over += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners else 0
	total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0

	goals_over += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0
	goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

	all_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0
	all_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0

In [36]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.004784688995215311
total_shots_over: 0.014354066985645933
total_shots_under: 0.9808612440191388


total_shots_on_target: 0.11004784688995216
total_shots_on_target_over: 0.35406698564593303
total_shots_on_target_under: 0.5358851674641149


total_booking_points: 0.1937799043062201
total_booking_points_over: 0.14354066985645933
total_booking_points_under: 0.6626794258373205


correct_score: 0.09569377990430622


winner: 0.39952153110047844


total_corners: 0.12440191387559808
total_corners_over: 0.32057416267942584
total_corners_under: 0.5550239234449761


total_fouls: 0.07177033492822966
total_fouls_over: 0.39952153110047844
total_fouls_under: 0.5287081339712919


goals_over: 0.2535885167464115
goals_under: 0.5334928229665071


all_over: 0.0023923444976076554
all_under: 0.1339712918660287


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [37]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(15, activation="relu", input_dim=15))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(14, activation="relu"))

	model.compile(
		optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learn_rate),
		loss="mse",
		metrics=["accuracy"])
	

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [38]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [39]:
# define a grid of the hyperparameter search space
hidden_layer_one = [10,20,30]
learn_rate = [1e-2, 1e-3, 1e-4]
dropout = [0.1, 0.2, 0.3]
batch_size = [16, 32, 64]
epochs = [10, 50, 100, 200]
n_h_layers = [1, 2, 3, 4]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [40]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	# estimator.save("../stats_regression_model.h5")
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [41]:
# searcher = GridSearchCV(estimator=model, n_jobs=-2, 
# 	param_grid=grid, scoring=scoring, verbose=4, cv=3, refit=True)

# searchResults = searcher.fit(X_train, y_train)

# bestScore = searchResults.best_score_
# bestParams = searchResults.best_params_
# print(f"[INFO] best score is {bestScore} using {bestParams}")

In [42]:
# tuned_model = searcher.best_params_
# tuned_model

In [43]:
hidden_layer_one = [10]
learn_rate = [1e-4]
dropout = [0.3]
batch_size = [64]
epochs = [10]
n_h_layers = [3]

model = get_mlp_model(hidden_layer_one=10, learn_rate=0.0001, dropout=0.3, n_h_layers=3)
model.fit(X_train, y_train, epochs=500, batch_size=64, )

model.save("../stats_regression_model.h5")
pd.DataFrame(data=components[:, [p for p in range(15)]], columns=pca.get_feature_names_out(), )

Epoch 1/500


27/27 [==============================] - 0s 495us/step - loss: 53.0407 - accuracy: 0.0060
Epoch 2/500
27/27 [==============================] - 0s 438us/step - loss: 52.8998 - accuracy: 0.0060
Epoch 3/500
27/27 [==============================] - 0s 432us/step - loss: 52.6398 - accuracy: 0.0084
Epoch 4/500
27/27 [==============================] - 0s 424us/step - loss: 52.4660 - accuracy: 0.0066
Epoch 5/500
27/27 [==============================] - 0s 432us/step - loss: 52.2252 - accuracy: 0.0096
Epoch 6/500
27/27 [==============================] - 0s 426us/step - loss: 51.9414 - accuracy: 0.0084
Epoch 7/500
27/27 [==============================] - 0s 2ms/step - loss: 51.5247 - accuracy: 0.0114
Epoch 8/500
27/27 [==============================] - 0s 444us/step - loss: 51.2346 - accuracy: 0.0168
Epoch 9/500
27/27 [==============================] - 0s 441us/step - loss: 50.8672 - accuracy: 0.0174
Epoch 10/500
27/27 [==============================] - 0s 420us/step - loss: 50.3563 - accuracy: 

,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,pca11,pca12,pca13,pca14
0,8.227741,0.768791,-1.230863,0.629500,-2.275751,0.834127,-0.495641,1.478604,0.907183,0.372550,-1.663873,-1.101019,0.351176,0.147980,0.062929
1,2.409746,1.643127,0.419988,1.420991,-0.601646,0.538918,1.043850,-0.477200,0.171570,-0.651725,0.752860,-0.161900,-0.286501,0.599887,0.266104
2,5.632586,-2.215035,-0.542818,1.479342,0.094398,0.113500,0.068508,-0.474223,0.213025,0.194923,0.309978,-0.458971,0.114207,0.065218,-0.262211
3,0.398500,1.592015,-1.622763,-1.230258,0.346584,1.291985,1.362661,1.121899,0.392571,-0.240816,1.320348,0.813986,-0.278462,-0.320303,0.144109
4,0.266204,-0.445595,1.728261,0.744165,0.761127,-1.452097,1.006612,-0.684164,-0.327990,0.546675,-0.125783,0.663222,-0.490705,1.287395,0.523730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,-6.358955,2.480779,1.664188,-0.809720,0.455320,0.158185,-0.147266,-0.346205,0.084570,0.198111,0.088984,-0.233710,-0.228670,-0.066363,-0.101494
2082,-3.196203,-3.356125,1.151455,1.284638,0.391472,-1.382331,-0.737146,-0.480154,1.127383,-0.728928,0.995593,1.253751,0.339998,-0.424561,-0.178794
2083,0.621913,6.741290,-1.769594,-1.676987,1.951403,1.492804,-0.341187,-0.621244,0.426888,0.483449,-0.877455,0.249528,-0.608699,0.309198,-0.339283
2084,2.642526,1.259347,-0.047434,-1.448993,-1.638753,-0.937967,0.619766,-0.413280,0.410560,0.115090,0.330088,-0.620822,-0.406892,0.107514,0.356692


In [44]:
print(len
	  (X_train))

1668


## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?